 <a name='ind'/>

# <ins>Indice</ins>:

- ## [Extracción](#ext)
- ## [Transformación](#trans)
- ## [Carga](#load)

In [44]:
import pandas as pd
from dateparser import parse
import requests as rq
import pymongo
from password import *   # Datso de la API de Google

<a name='ext'/>

###### ⬆️ [Indice](#ind)

### Extracción
***

Vamos a enriquecer la infomración obtenida en el [proyecto](https://github.com/gusavato/3_IRONHACK_ETL) anterior de extracción de datos. Partiremos del archivo .parquet donde se alberga la información de los distintos festivales que se celebran en el año 2023

In [30]:
fest = pd.read_parquet('../data/df_fest_clean.parquet')

In [31]:
fest.head()

,Festival,url,Cartel,Start_Date,End_Date,Location,info
0,Mallorca Live Festival 2023,https://www.songkick.com/es/festivals/1520684-...,"[Black Eyed Peas, Chemical Brothers, The Dandy...",jueves 18 mayo 2023,sábado 20 mayo 2023,Antic Aquapark,"Camí Cala Figuera, 1\n07181\nCalvia, Spain\nww..."
1,Primavera a la Ciutat 2023,https://www.songkick.com/es/festivals/3452756-...,"[Jake Bugg, Pet Shop Boys, Confidence Man, La ...",martes 30 mayo 2023,miércoles 31 mayo 2023,Sala Apolo,"C/ Nou De La Rambla 113\n08004\nBarcelona, Spa..."
2,Primavera In The City - Madrid 2023,https://www.songkick.com/es/festivals/3500869-...,"[Bleachers, Julia Jacklin, PUP, Black Country,...",lunes 05 junio 2023,domingo 11 junio 2023,Unknown,Unknown
3,Primavera Sound Festival 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",jueves 08 junio 2023,domingo 11 junio 2023,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
4,Primavera Pack 2023,https://www.songkick.com/es/festivals/3495900-...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",jueves 08 junio 2023,sábado 10 junio 2023,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."


<a name='trans'/>

###### ⬆️ [Indice](#ind)

### Transformación
***

El primer paso que vamos a realizar es el de formatear las fechas de inicio y final de festival a un formato correcto (actualmemte string). Para ello nos ayudamos de la librería dateparser

In [32]:
fest.Start_Date = fest.Start_Date.apply(lambda x: parse(x,languages=['es']))
fest.End_Date = fest.End_Date.apply(lambda x: parse(x,languages=['es']))

In [33]:
fest.head()

,Festival,url,Cartel,Start_Date,End_Date,Location,info
0,Mallorca Live Festival 2023,https://www.songkick.com/es/festivals/1520684-...,"[Black Eyed Peas, Chemical Brothers, The Dandy...",2023-05-18,2023-05-20,Antic Aquapark,"Camí Cala Figuera, 1\n07181\nCalvia, Spain\nww..."
1,Primavera a la Ciutat 2023,https://www.songkick.com/es/festivals/3452756-...,"[Jake Bugg, Pet Shop Boys, Confidence Man, La ...",2023-05-30,2023-05-31,Sala Apolo,"C/ Nou De La Rambla 113\n08004\nBarcelona, Spa..."
2,Primavera In The City - Madrid 2023,https://www.songkick.com/es/festivals/3500869-...,"[Bleachers, Julia Jacklin, PUP, Black Country,...",2023-06-05,2023-06-11,Unknown,Unknown
3,Primavera Sound Festival 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-11,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
4,Primavera Pack 2023,https://www.songkick.com/es/festivals/3495900-...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-10,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."


El siguiente paso es conseguir las coordenadas de longitud y latitud de cada uno de los festivales. En la columna info podemos obtener esta información

In [34]:
# Eliminamos los saltos de linea de la columna info

fest['info'] = fest['info'].apply(lambda x :' '.join(x.split()))

In [23]:
def get_coordinates(x):
    """
    Función que dada una query, se envía la query a google maps, y recoge
    la información de latitud y logintud

    """
    global API_KEY_GOOGLE
    address = x.replace(' ', '%20')
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY_GOOGLE}'
    try:
        response = rq.get(url)
        lat, long = response.json()['results'][0]['geometry']['location'].values()
        country = response.json()['results'][0]['address_components'][-2]['long_name']
        return {'Latitud': lat,'Longitud': long}
    except:
        return {'Latitud': None,'Longitud': None}

In [24]:
coord = [get_coordinates(query) for query in fest['info']]

In [27]:
pd.DataFrame(coord)

,Latitud,Longitud
0,39.506894,2.519330
1,41.374302,2.169684
2,NaN,NaN
3,39.470690,-0.427288
4,39.470690,-0.427288
...,...,...
475,41.887226,-8.786437
476,41.171382,-8.676519
477,38.698719,-9.238162
478,47.553085,19.048025


In [35]:
fest = pd.concat([fest,pd.DataFrame(coord)],axis=1)
fest.head()

,Festival,url,Cartel,Start_Date,End_Date,Location,info,Latitud,Longitud
0,Mallorca Live Festival 2023,https://www.songkick.com/es/festivals/1520684-...,"[Black Eyed Peas, Chemical Brothers, The Dandy...",2023-05-18,2023-05-20,Antic Aquapark,"Camí Cala Figuera, 1 07181 Calvia, Spain www.m...",39.506894,2.519330
1,Primavera a la Ciutat 2023,https://www.songkick.com/es/festivals/3452756-...,"[Jake Bugg, Pet Shop Boys, Confidence Man, La ...",2023-05-30,2023-05-31,Sala Apolo,"C/ Nou De La Rambla 113 08004 Barcelona, Spain...",41.374302,2.169684
2,Primavera In The City - Madrid 2023,https://www.songkick.com/es/festivals/3500869-...,"[Bleachers, Julia Jacklin, PUP, Black Country,...",2023-06-05,2023-06-11,Unknown,Unknown,NaN,NaN
3,Primavera Sound Festival 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-11,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35 28...",39.470690,-0.427288
4,Primavera Pack 2023,https://www.songkick.com/es/festivals/3495900-...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-10,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35 28...",39.470690,-0.427288


In [40]:
# Guardamos la información de aquellos festivales que no se ha localizado la información

fest[fest.Longitud.isna()].Festival.to_csv('../data/fest_wo_loc.csv', index=False)

Guardamos la información en un archivo parquet nuevo. No hemos borrado los nulos ya que estos solo afectaran a la hora de ubicar el festival en un mapa, y no los hemos transformado a un valor númerico 0, porque la ubicación sería erronea. Si lo dejamos como None, podemos configurar la visualización para que no tenga en cuenta estos datos

In [42]:
fest.to_parquet('../data/fest_coord.parquet',index=False)

<a name='load'/>

###### ⬆️ [Indice](#ind)

### Carga
***

In [ ]:
cursor = pymongo.MongoClient('mongodb://localhost:27017')

In [ ]:
db_fest = cursor.Festivales

In [ ]:
fest_coord = db_fest.fest_coord

In [ ]:
fest.Cartel = fest.Cartel.apply(list)    # Convertimos el array de np a lista para no tener problema con la subida de datos a la DB 
fest_coord.insert_many(fest.to_dict(orient='records'))